In [1]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow
from sklearn.metrics import mean_squared_error
import pandas as pd
from datetime import datetime
import pickle

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
# client.list_experiments() # In older versions

client.search_experiments() # Ine the new version

[<Experiment: artifact_location='/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment Tracking/mlruns/2', creation_time=1748234642656, experiment_id='2', last_update_time=1748234642656, lifecycle_stage='active', name='my-cool-exp', tags={}>,
 <Experiment: artifact_location='/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment Tracking/mlruns/1', creation_time=1748193104587, experiment_id='1', last_update_time=1748193104587, lifecycle_stage='active', name='nyc-taxi', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748192589586, experiment_id='0', last_update_time=1748192589586, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name="new-exp")

'3'

In [5]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 12",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: d3fad227b05b46b7b1ca48210d85916b, rmse: 6.3134
run id: 04acdf5182b5406284accc98b4ff5708, rmse: 6.3134
run id: 33a6910f136c4da384bb0df7ac7e96e0, rmse: 6.3134
run id: a472584a598f46cb8b1b42752a97abec, rmse: 6.3134
run id: f2c1269c03b84dd7800f873dbb860712, rmse: 6.3297
run id: 16b8a95d203d4ca8b38dbb726c09ac6f, rmse: 6.4126
run id: 213bd8a8b3754a289589313c3268aeae, rmse: 6.5648
run id: 638481b288074aa4bdc179c4f9b764d9, rmse: 11.1673


In [6]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [7]:
run_id = "213bd8a8b3754a289589313c3268aeae"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name='nyc-taxi')

Registered model 'nyc-taxi' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi'.


<ModelVersion: aliases=[], creation_timestamp=1748241211723, current_stage='None', description=None, last_updated_timestamp=1748241211723, name='nyc-taxi', run_id='213bd8a8b3754a289589313c3268aeae', run_link=None, source=('/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment '
 'Tracking/mlruns/1/213bd8a8b3754a289589313c3268aeae/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=4>

In [8]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1748233621639, description='', last_updated_timestamp=1748241211723, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748241211723, current_stage='None', description=None, last_updated_timestamp=1748241211723, name='nyc-taxi', run_id='213bd8a8b3754a289589313c3268aeae', run_link=None, source=('/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment '
  'Tracking/mlruns/1/213bd8a8b3754a289589313c3268aeae/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=4>,
  <ModelVersion: aliases=[], creation_timestamp=1748233681357, current_stage='Production', description='', last_updated_timestamp=1748235825050, name='nyc-taxi', run_id='04acdf5182b5406284accc98b4ff5708', run_link='', source=('/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment '
  'Tracking/mlruns/1/04acdf5182b5406284accc98b4ff5708/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=2>,
  <Model

In [9]:
model_name = 'nyc-taxi'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None
version: 2, stage: Production
version: 3, stage: Staging


/tmp/ipykernel_7810/1455972288.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [10]:
model_version = 3
new_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_7810/2949610654.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1748235408631, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2025-05-26', last_updated_timestamp=1748241211779, name='nyc-taxi', run_id='213bd8a8b3754a289589313c3268aeae', run_link=None, source=('/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment '
 'Tracking/mlruns/1/213bd8a8b3754a289589313c3268aeae/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [11]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1748233621639, description='', last_updated_timestamp=1748241211779, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748241211723, current_stage='None', description=None, last_updated_timestamp=1748241211723, name='nyc-taxi', run_id='213bd8a8b3754a289589313c3268aeae', run_link=None, source=('/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment '
  'Tracking/mlruns/1/213bd8a8b3754a289589313c3268aeae/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=4>,
  <ModelVersion: aliases=[], creation_timestamp=1748233681357, current_stage='Production', description='', last_updated_timestamp=1748235825050, name='nyc-taxi', run_id='04acdf5182b5406284accc98b4ff5708', run_link='', source=('/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment '
  'Tracking/mlruns/1/04acdf5182b5406284accc98b4ff5708/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=2>,
  <Model

In [12]:
model_name = 'nyc-taxi'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None
version: 2, stage: Production
version: 3, stage: Staging


/tmp/ipykernel_7810/1455972288.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [13]:
date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1748235408631, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2025-05-26', last_updated_timestamp=1748241211831, name='nyc-taxi', run_id='213bd8a8b3754a289589313c3268aeae', run_link=None, source=('/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment '
 'Tracking/mlruns/1/213bd8a8b3754a289589313c3268aeae/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [14]:
def read_data_frame(file):
    df = pd.read_parquet(file)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    
    return dv.transform(train_dicts)

def test_model(name, stage, x_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(x_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [15]:
df = read_data_frame('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet')

In [16]:
run_id = 'd3fad227b05b46b7b1ca48210d85916b'
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/MLOps-Zoomcamp/2. Module 2 - Experiment Tracking/preprocessor'

In [17]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [18]:
x_test = preprocess(df, dv[0])

In [19]:
target = "duration"
y_test = df[target].values

In [20]:
%time test_model(name=model_name, stage="Production", x_test=x_test, y_test=y_test)

CPU times: user 12.5 s, sys: 67.4 ms, total: 12.6 s
Wall time: 7.45 s


{'rmse': 6.266264885907768}